Setting Up your enviroment and connect APIs

In [89]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import ebooklib
from ebooklib import epub

In [90]:
from openaikey import *

import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

Reading PDF

In [91]:
#The object that has all pdf content
the_reader = PdfReader('/Users/naelshaker/Work/Langchain to read pdf/Hooked How to Build Habit-Forming Products • Supplemental Workbook (Nir Eyal, Ryan Hoover) (z-lib.org).pdf')

In [92]:
print(type(the_reader))

<class 'PyPDF2._reader.PdfReader'>


Trying to read epub

In [93]:
book = epub.read_epub('/Users/naelshaker/Work/Langchain to read pdf/Inspired_ How to Create Tech Products Customers Love by Marty Cagan.epub')


/Users/naelshaker/opt/anaconda3/lib/python3.9/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


Get the number of pages

Method 1

In [94]:
# # Get the number of pages in the PDF file
# num_pages = len(the_reader.pages)
# # len(reader.pages)
# print(num_pages)

# # Get the text content of each page in the PDF file
# for page_num in range(num_pages):
#     page = pdf_reader.getPage(page_num)
#     page_content = page.extractText()
#     print(page_content)

Method 2

In [95]:
raw_text = ''

#getting text per page
for i, page in enumerate(the_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text
        

raw_text


"HOOKED WORKBOOK | CHAPTER X1Copyright © 2014 Nir Eyal, NirAndFar.com. All rights reserved.  \n  \nSUPPLEMENTAL\nWORKBOOKSPECIAL OFFER\nTESTIMONIALSBuy Hooked now and receive a special book bundle. You’ll receive four valuable resources to \nhelp you build habits in yourself and your user\nFirst, buy your copy of Hooked by clicking a retailer below, then visit:\nwww.hookmodel.com/#special-offer\nAmazon.com Barnes & Nobles IndieBound.org\n“You’ll read this. Then you’ll hope your \ncompetition isn’t reading this. \nIt’s that good. ”“When it comes to driving engagement \nand building habits, Hooked is an \nexcellent guide into the mind of the \nuser. ”“Nir’s work is an essential crib sheet for \nany startup looking to understand user \npsychology. ”Stephen P . Anderson\nAuthor of “Seductive Interaction Design”\n \n \n \nAndrew Chen\nTechnology Writer & InvestorDave McClure\nFounder of 500 startups\n                                       “This is the absolute best book on product developme

Reading the epub

In [96]:
for item in book.get_items():
    print(item.get_type())


10
1
1
1
2
2
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
4


In [97]:
epub_text = ''
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        content = item.get_content()
        epub_text += content.decode('utf-8')
        
print(epub_text)

<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">
  <head/>
  <body><section epub:type="cover">&#13;
<span epub:type="pagebreak" id="Page_C1" title="C1"/>&#13;
<figure><a id="coverstart"/><img alt="Cover Page" src="images/9781119387565.jpg"/></figure>&#13;
</section>&#13;
</body>
</html>
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">
  <head/>
  <body><section>&#13;
<nav class="tocList" epub:type="toc" id="toc">&#13;
<h1><b>Table of Contents</b></h1>&#13;
<ol>&#13;
<li class="contentsH1"><a href="cover.xhtml">Cover</a></li>&#13;
<li class="contentsH1"><a href="title.xhtml">Title Page</a></li>&#13;
<li class="contentsH

In [98]:
#splitting text each chunk has 1000 characters to not hit the token limit and the overlap 200 means 
# first sentence will have 1000 characters the second chunk will start from the 800th
text_splitter = CharacterTextSplitter(
    separator= '\n',
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
)

text = text_splitter.split_text(epub_text)

#to understand the overlap uncomment the next code 
len(text)
text[800]
# text[1]

'<li id="c64-li-0017">Good teams celebrate when they achieve a significant impact to the business results. Bad teams celebrate when they finally release something.</li>&#13;\n</ul>&#13;\n<p id="c64-para-0006w">If a significant number of these items strike too close to home, I hope you\'ll consider raising the bar for your team. See if you can\'t use the techniques in this book to experience the difference.</p>&#13;\n</section>&#13;\n</section>\n  </body>\n</html>\n<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body>\n    <section epub:type="chapter">&#13;\n<header>&#13;\n<h1><span epub:type="pagebreak" id="Page_315" title="315"/><a id="c65"/><span class="chapterNumber">CHAPTER 65</span><br/><span class="chapterTitle">Top Reasons for Loss of Innovation</span></h1></header>&#13;'

In [99]:
#download embeddings from openai
embeddings = OpenAIEmbeddings()

#adding it to vector database
doc_search = FAISS.from_texts(text, embeddings)
doc_search

In [100]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [101]:
chain = load_qa_chain(OpenAI(),chain_type= 'stuff')

Some Questions to test

In [105]:
query = "What was the cost of training the GPT4all article?"
docs = doc_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

" I don't know."

In [103]:
query = "What are the hook model steps?"
docs = doc_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

" I don't know."

In [109]:
query = "WHat is the right way to do the roadmap?"
docs = doc_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

' An outcome-based roadmap, where each item is stated as a business problem to solve, rather than the feature or project that may or may not solve it. This way, the product team is solving business problems rather than building features.'